In [0]:
pip install geopy

Python interpreter will be restarted.
  Using cached geopy-2.4.1-py3-none-any.whl (125 kB)
  Using cached geographiclib-2.0-py3-none-any.whl (40 kB)
Python interpreter will be restarted.


In [0]:
from pyspark.sql.types import *
from pyspark.sql import DataFrame
from functools import reduce
from pyspark.sql.functions import *
from pyspark.sql import functions as F
import os
from pyspark.sql import *
from pyspark.sql.functions import col, sum
import geopy as Photon

In [0]:
%run "/Users/vaishnavi.balureddy2@cognizant.com/Uber Pick Ups/Formatting Silver Dataframes"

Out[5]: False

Out[9]: 18

No of rows '100':
DataFrame for table 'other_American_B01362':
+----------+-----------+--------------------+--------------------+
|      DATE|       Time|     PICK_UP_ADDRESS|            Category|
+----------+-----------+--------------------+--------------------+
|07-01-2014|12:00:00 AM| 874 E 139th St M...|other_American_B0...|
|07-01-2014|12:01:00 AM| 628 E 141st St M...|other_American_B0...|
+----------+-----------+--------------------+--------------------+
only showing top 2 rows

No of rows '100':
DataFrame for table 'other_Diplo_B01196':
+----------+-----------+--------------------+------------------+
|      Date|       Time|          PU_Address|          Category|
+----------+-----------+--------------------+------------------+
|07-01-2014|12:00:00 AM| 2396 Valentine A...|other_Diplo_B01196|
|07-01-2014|12:01:00 AM| 1859 Walton Ave ...|other_Diplo_B01196|
+----------+-----------+--------------------+------------------+
only showing top 2 rows

No of rows '100':
DataFrame for tab

Out[11]: DataFrame[key: string, value: string]

In [0]:
spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY")

Out[37]: DataFrame[key: string, value: string]

In [0]:
schema1 = StructType([
    StructField("Date", StringType(), True),
    StructField("Time", StringType(), True),
    StructField("Lat", DoubleType(), True),
    StructField("Lon", DoubleType(), True),
    StructField("Dispatching_Base_Num", StringType(), True),
    StructField("Category", StringType(), True),
])


schema2 = StructType([
    StructField("Date", StringType(), True),
    StructField("Dispatching_Base_Num", StringType(), True),
    StructField("Category", StringType(), True),
    StructField("Active_Vehicles", IntegerType(), True),
    StructField("Base_Name", StringType(), True),
    StructField("Trips", IntegerType(), True)
])

schema3 = StructType([
    StructField("Date", StringType(), True),
    StructField("Time", StringType(), True),
    StructField("Pickup_Address", StringType(), True),
    StructField("Category", StringType(), True)
])

schema4 = StructType([
    StructField("Date", StringType(), True),
    StructField("Time", StringType(), True),
    StructField("Dispatching_Base_Num", StringType(), True),
    StructField("Category", StringType(), True)
])

In [0]:
dfs1 = {
    "uber_raw_data_apr14": dfs["uber_raw_data_apr14"],
    "uber_raw_data_may14": dfs["uber_raw_data_may14"],
    "uber_raw_data_jun14": dfs["uber_raw_data_jun14"],
    "uber_raw_data_jul14": dfs["uber_raw_data_jul14"],
    "uber_raw_data_aug14": dfs["uber_raw_data_aug14"],
    "uber_raw_data_sep14": dfs["uber_raw_data_sep14"]
}

dfs2 = {
    "Uber_Jan_Feb_FOIL": dfs["Uber_Jan_Feb_FOIL"],
    "other_FHV_services_jan_aug_2015": dfs["other_FHV_services_jan_aug_2015"]
}

dfs3 = {
    "other_Skyline_B00111": dfs["other_Skyline_B00111"],
    "other_Firstclass_B01536": dfs["other_Firstclass_B01536"],
    "other_American_B01362": dfs["other_American_B01362"],
    "other_Highclass_B01717": dfs["other_Highclass_B01717"],
    "other_Prestige_B01338": dfs["other_Prestige_B01338"],
    "other_Federal_02216": dfs["other_Federal_02216"], 
    "other_Carmel_B00256": dfs["other_Carmel_B00256"],
    "other_Lyft_B02510": dfs["other_Lyft_B02510"]
}

dfs4 = {
    "uber_raw_data_janjune_15": dfs["uber_raw_data_janjune_15"]
}


In [0]:
combined_df = spark.createDataFrame([], schema=schema1)

for table_name, df in dfs1.items():

    try:
        combined_df = combined_df.unionByName(df)
    except Exception as e:
        print(f"Error occurred with DataFrame: {table_name}")
        print(e)
        break

In [0]:
Uber_AprSep14 = combined_df.withColumn('timestamp_string',to_timestamp('Time')) 
Uber_AprSep14 = Uber_AprSep14.withColumn("timestamp_string1", split("timestamp_string", " ")[1])

Uber_AprSep14 = Uber_AprSep14.drop('Time')
Uber_AprSep14 = Uber_AprSep14.withColumnRenamed("timestamp_string1", "Time")
Uber_AprSep14 = Uber_AprSep14.drop('timestamp_string')
Uber_AprSep14 = Uber_AprSep14.drop('timestamp_string1')

Uber_AprSep14 = Uber_AprSep14.withColumn("trip_ID", concat(lit('1'), lpad((monotonically_increasing_id() % 100000000).cast('string'), 7, '0'))) 
Uber_AprSep14.show(5)

+----------+-------+--------+--------------------+-------------------+--------+--------+
|      Date|    Lat|     Lon|Dispatching_Base_Num|           Category|    Time| trip_ID|
+----------+-------+--------+--------------------+-------------------+--------+--------+
|2014-04-01| 40.769|-73.9549|              B02512|uber_raw_data_apr14|00:11:00|11947673|
|2014-04-01|40.7267|-74.0345|              B02512|uber_raw_data_apr14|00:17:00|11947673|
|2014-04-01|40.7316|-73.9873|              B02512|uber_raw_data_apr14|00:21:00|11947673|
|2014-04-01|40.7588|-73.9776|              B02512|uber_raw_data_apr14|00:28:00|11947673|
|2014-04-01|40.7594|-73.9722|              B02512|uber_raw_data_apr14|00:33:00|11947674|
+----------+-------+--------+--------------------+-------------------+--------+--------+
only showing top 5 rows



In [0]:
all_columns = set(["Date","Dispatching_Base_Num","Category","Active_Vehicles","Base_Name","Trips"])
combined_df = spark.createDataFrame([], schema=schema2)

# Iterate over the existing DataFrames and add columns to the combined DataFrame
for table_name, df in dfs2.items():
    existing_columns = set(df.columns)
    missing_columns = all_columns - existing_columns
    
    # Add missing columns with null values
    for col in missing_columns:
        df = df.withColumn(col, lit("Uber"))
    #Remove white spaces in column names to resolve columns
    tempList = [] 
    for col in df.columns:
        new_name = col.strip()
        new_name = "".join(new_name.split())
        new_name = new_name.replace('.','') 
        tempList.append(new_name)

    try:
        combined_df = combined_df.unionByName(df)
    except Exception as e:
        print(f"Error occurred with DataFrame: {table_name}")
        print(e)
        break

In [0]:
UBERandFHV = combined_df
UBERandFHV = UBERandFHV.withColumn("trip_ID", concat(lit('2'), lpad((monotonically_increasing_id() % 100000000).cast('string'), 7, '0')))
UBERandFHV.tail(5)

Out[43]: [Row(Date='01-02-2015', Dispatching_Base_Num='B01777', Category='other_FHV_services_jan_aug_2015', Active_Vehicles='131', Base_Name='PREMIUM RADIO DISP & MULTI SVC', Trips='677', trip_ID='29411423'),
 Row(Date='01-02-2015', Dispatching_Base_Num='B01848', Category='other_FHV_services_jan_aug_2015', Active_Vehicles=' -   ', Base_Name='GRAND LIMO & CAR SERVICE INC.', Trips='11', trip_ID='29411424'),
 Row(Date='01-02-2015', Dispatching_Base_Num='B01877', Category='other_FHV_services_jan_aug_2015', Active_Vehicles='43', Base_Name='U.S.A. LIMO. INC.', Trips='76', trip_ID='29411425'),
 Row(Date='01-02-2015', Dispatching_Base_Num='B01899', Category='other_FHV_services_jan_aug_2015', Active_Vehicles='88', Base_Name='ALLSTATE PRIVATE CAR & LIMO,INC', Trips='726', trip_ID='29411426'),
 Row(Date='01-02-2015', Dispatching_Base_Num='B01949', Category='other_FHV_services_jan_aug_2015', Active_Vehicles='1', Base_Name='FIRST CLASS OF NEW YORK INC.', Trips='1', trip_ID='29411427')]

In [0]:
unique_categories = UBERandFHV.select('Dispatching_Base_Num').distinct().collect()

print("Unique categories in UBERandFHV:")
for row in unique_categories:
    print(row['Dispatching_Base_Num'])
UBERandFHV.show()

Unique categories in UBERandFHV:
B02512
B02598
B02682
B02765
B02617
B02764
B01341
B00887
B02715
B00948
B01197
B00029
B01313
B01386
B00381
B00254
B00837
B02003
B01087
B00221
B01300
B01285
B02513
B01367    
B02569
B02653
B02566
B02022
B01899
B01848
B01083
B00412
B01990
B01282
B00310
B02285
B00013
B00227
B00457
B01312
B01148
B01522
B01066
B01651
B00014
B00053
B00919
B02475
B02147
B00477
b01346
B01777
B01877
B02120
B01315
B00280
B02107
B00552
B00623
B00248
B00095
B01949
B02096
B01984
+----------+--------------------+-----------------+---------------+---------+-----+--------+
|      Date|Dispatching_Base_Num|         Category|Active_Vehicles|Base_Name|Trips| trip_ID|
+----------+--------------------+-----------------+---------------+---------+-----+--------+
|01-01-2015|              B02512|Uber_Jan_Feb_FOIL|            190|     Uber| 1132|21947673|
|01-01-2015|              B02765|Uber_Jan_Feb_FOIL|            225|     Uber| 1765|21947673|
|01-01-2015|              B02764|Uber_Jan_Feb_FOIL

In [0]:
combined_df = spark.createDataFrame([], schema=schema3)

for table_name, df in dfs3.items():

    try:
        combined_df = combined_df.unionByName(df)
    except Exception as e:
        print(f"Error occurred with DataFrame: {table_name}")
        print(e)
        break

In [0]:
FHV_Data = combined_df.withColumn('timestamp_string',to_timestamp('Time')) 
FHV_Data = FHV_Data.withColumn("timestamp_string1", split("timestamp_string", " ")[1])

FHV_Data = FHV_Data.drop('Time')
FHV_Data = FHV_Data.withColumnRenamed("timestamp_string1", "Time")
FHV_Data = FHV_Data.drop('timestamp_string')
FHV_Data = FHV_Data.drop('timestamp_string1')

FHV_Data = FHV_Data.withColumn("trip_ID", concat(lit('3'), lpad((monotonically_increasing_id() % 100000000).cast('string'), 7, '0')))
                
FHV_Data.show(5)

+----------+--------------------+--------------------+--------+--------+
|      Date|      Pickup_Address|            Category|    Time| trip_ID|
+----------+--------------------+--------------------+--------+--------+
|07-01-2014|   622 THIRD AV M   |other_Skyline_B00111|20:27:00|31947673|
|07-01-2014|    E 77TH ST   M   |other_Skyline_B00111|21:04:00|31947673|
|07-01-2014|67 WEST PALISADES...|other_Skyline_B00111|22:20:00|31947673|
|07-01-2014|130 MIDDLE NECK R...|other_Skyline_B00111|12:28:00|31947673|
|07-01-2014|   36 E 31ST ST M   |other_Skyline_B00111|16:45:00|31947674|
+----------+--------------------+--------------------+--------+--------+
only showing top 5 rows



In [0]:
distinct_pickup_addresses_count = FHV_Data.select('Pickup_Address').distinct().count()
print("Number of distinct pickup addresses:", distinct_pickup_addresses_count)

Number of distinct pickup addresses: 641


In [0]:
Uber_JanJun15 = dfs['uber_raw_data_janjune_15'].withColumn('timestamp_string',to_timestamp('Time')) 
Uber_JanJun15 = Uber_JanJun15.withColumn("timestamp_string1", split("timestamp_string", " ")[1])

Uber_JanJun15 = Uber_JanJun15.drop('Time')
Uber_JanJun15 = Uber_JanJun15.withColumnRenamed("timestamp_string1", "Time")
Uber_JanJun15 = Uber_JanJun15.drop('timestamp_string')
Uber_JanJun15 = Uber_JanJun15.drop('timestamp_string1')

Uber_JanJun15 = Uber_JanJun15.withColumn("trip_ID", concat(lit('4'), lpad((monotonically_increasing_id() % 100000000).cast('string'), 7, '0')))
                
Uber_JanJun15.show(5)

+--------------------+--------------------+----------+--------+--------+
|Dispatching_Base_Num|            Category|      Date|    Time| trip_ID|
+--------------------+--------------------+----------+--------+--------+
|              B02617|uber_raw_data_jan...|2015-05-17|09:47:00|40000000|
|              B02617|uber_raw_data_jan...|2015-05-17|09:47:00|40000001|
|              B02617|uber_raw_data_jan...|2015-05-17|09:47:00|40000002|
|              B02617|uber_raw_data_jan...|2015-05-17|09:47:00|40000003|
|              B02617|uber_raw_data_jan...|2015-05-17|09:47:00|40000004|
+--------------------+--------------------+----------+--------+--------+
only showing top 5 rows



In [0]:
base_name_mapping = {
    'B02512': 'Unter',
    'B02598': 'Hinter',
    'B02617': 'Weiter',
    'B02682': 'Schmecken',
    'B02764': 'Danach-NY',
    'B02765': 'Grun',
    'B02835': 'Dreist',
    'B02836': 'Drinnen'
}

def get_base_name(base_num):
    return base_name_mapping.get(base_num, 'Unknown')
get_base_name_udf = udf(get_base_name, StringType())



Uber_AprSep14 = Uber_AprSep14.withColumn("Base_Name", get_base_name_udf(Uber_AprSep14["Dispatching_Base_Num"]))

In [0]:
Uber_AprSep14.show(2)

+----------+-------+--------+--------------------+-------------------+--------+--------+---------+
|      Date|    Lat|     Lon|Dispatching_Base_Num|           Category|    Time| trip_ID|Base_Name|
+----------+-------+--------+--------------------+-------------------+--------+--------+---------+
|2014-04-01| 40.769|-73.9549|              B02512|uber_raw_data_apr14|00:11:00|11947673|    Unter|
|2014-04-01|40.7267|-74.0345|              B02512|uber_raw_data_apr14|00:17:00|11947673|    Unter|
+----------+-------+--------+--------------------+-------------------+--------+--------+---------+
only showing top 2 rows



In [0]:

Uber_JanJun15 = Uber_JanJun15.withColumn("Base_Name", get_base_name_udf(Uber_JanJun15["Dispatching_Base_Num"]))
UBERandFHV = UBERandFHV.withColumn("Base_Name", get_base_name_udf(UBERandFHV["Dispatching_Base_Num"]))

In [0]:
Uber_JanJun15 = Uber_JanJun15.withColumn("Price[$]", (rand() * (100 - 5) + 5).cast("double"))
Uber_JanJun15 = Uber_JanJun15.withColumn("Price[$]", format_number("Price[$]", 2))

Writing these silver tables to tables in catalog. Run the next cells or the notebook

In [0]:
new_folder_name = "Silver_Tables"
new_folder_path = "dbfs:/FileStore/UberPickUps/" + new_folder_name + "/"
dbutils.fs.mkdirs(new_folder_path)

try:
    dbutils.fs.mkdirs(new_folder_path)
    print(f"Folder '{new_folder_name}' created successfully at path '{new_folder_path}'.")
except Exception as e:
    print(f"Failed to create folder '{new_folder_name}' at path '{new_folder_path}'.")
    print(f"Error: {e}")

Folder 'Silver_Tables' created successfully at path 'dbfs:/FileStore/UberPickUps/Silver_Tables/'.


In [0]:
create_db_qry=f"""CREATE DATABASE IF NOT EXISTS Silver"""
spark.sql(create_db_qry)

Out[54]: DataFrame[]

In [0]:

path =  "dbfs:/FileStore/UberPickUps/Silver_Tables/Uber_AprSep14/"
Uber_AprSep14.write.format("delta").mode("overwrite").option("path",path).save()
Uber_AprSep14.write.mode('append').saveAsTable('Silver.Uber_AprSep14')

In [0]:
Uber_JanJun15.write.format("delta").mode("overwrite").option("path", "dbfs:/FileStore/UberPickUps/Silver_Tables/Uber_JanJun15/").save()
Uber_JanJun15.write.mode('append').saveAsTable('Silver.Uber_JanJun15')

In [0]:
UBERandFHV.write.format("delta").mode("overwrite").option("path", "dbfs:/FileStore/UberPickUps/Silver_Tables/UBERandFHV/").save()
UBERandFHV.write.mode('append').saveAsTable('Silver.UBERandFHV')

In [0]:
FHV_Data.write.format("delta").mode("overwrite").option("path", "dbfs:/FileStore/UberPickUps/Silver_Tables/FHV_Data/").save()
FHV_Data.write.mode('append').saveAsTable('Silver.FHV_Data')